# Processo de geração e avaliação de resposta sobre design de visualização de dados com LLMs

O processo de geração é algo que pode ser divido em algumas partes que ajudam a construir o produto final a ser coletado.

Para este trabalho, o objetivo é utilizar o grande modelo de linguagem da empresa OpenAI para gerar perguntas e respostas que auxiliem no processo de design de visualização de dados.

### Cenário

Imagine o seguinte cenário:

<i>Você é gerente de uma empresa de venda de eletronicos e que tem lojas espalhadas por todo o Brasil. O seu gerente do setor de smartphones, acompanhando os lançamentos dos dispositivos, avisa que será lançado um novo dispositivo Apple. Para distribuir os novos dispositivos, o seu gerente deverá apresentar quais lojas vendem mais, assim você poderá mandar mais dispositivos para o estoque.</i>



Desejos na visualização:
- Usuário: Gerente de tecnologia.
- Desejo: Mostrar as lojas que mais vendem smartphones Apple durante o ultimo ano

Entendendo esses pontos, é possível que sejam construídas respostas e avaliação das respostas com o modelo. O intuíto é que entendo o contexto do usuário e os desejos dele o sistema possa auxiliar no design da visualização.

In [3]:
usuario = "gerente de tecnologia"
desejo = "Mostrar as lojas que mais vendem smartphones Apple durante o ultimo ano"

Vamos agora montar o prompt com o perfil do usuário e o desejo que quer apresentar com a visualização

In [4]:
def criarPrompt (usuario, desejo):
    return "Eu, como um " + usuario.lower() + ", preciso " + desejo.lower()+ ", quais poderiam ser as melhores visualizações de dados para isso?"

promptPergunta = criarPrompt(usuario, desejo)
print(promptPergunta)

Eu, como um gerente de tecnologia, quero mostrar as lojas que mais vendem smartphones apple durante o ultimo ano, quais poderiam ser as melhores visualizações de dados para isso?


### Lançar uma pergunta para o modelo

Tendo o prompt construído, é possíve realizar o primeiro pedido ao modelo, neste caso o ChatGPT. Com esse prompt podemos coleta uma lista com as sugestões de visualização de dados que o modelo acredita que poderia suprir a necessidade do usuário.

- Importando o ChatGPT

In [5]:
from openai import OpenAI # type: ignore
client = OpenAI()

- Denfinindo a função do assistente para pergunta

In [6]:
pergunta = [
    {
        # aqui definimos que a resposta deve vir no formato JSON
        "role": "system",
        "content": "Você, como um assistente no processo de design de visualização de dados, deverá responder a pergunta com uma lista de possibilidades de forma de visualização de dados como resposta e retornar em uma lista no formato JSON"
    },
    {
        # aqui definimos o padrão de resposta em cada item da lista
        "role": "user",
        "content": "Cada item da lista de sugestões deve vir no formato: {/tipo: [tipo da visualuação de dados], descrição: [descrição clara do motivo de usar com pelo menos 50 palavras], variaveis: [lista com as variáveis necessárias para usar essa visualização e como usar a variável] }/"
    },
    {
        # aqui definimos que a pergunta como a linha escrita no prompt
        "role": "user",
        "content": promptPergunta
     }
]
print(pergunta)

[{'role': 'system', 'content': 'Você como um assistente no processo de design de visualização de dados, deverá responder a pergunta com uma lista de possibilidades de forma de visualização de dados como resposta e retornar em uma lista no formato JSON'}, {'role': 'user', 'content': 'Cada item da lista de sugestões deve vir no formato: {/tipo: [tipo da visualuação de dados], descrição: [descrição clara do motivo de usar com pelo menos 50 palavras], variaveis: [lista com as variáveis necessárias para usar essa visualização e como usar a variável] }/'}, {'role': 'user', 'content': 'Eu, como um gerente de tecnologia, quero mostrar as lojas que mais vendem smartphones apple durante o ultimo ano, quais poderiam ser as melhores visualizações de dados para isso?'}]


- Criando uma função para executar o Chat

In [7]:
def executarChat(mensagem):
  return client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    response_format={ "type": "json_object" },
    messages=mensagem,
  )

- Executando o chat com o modelo e o formato para pergunta

In [12]:
responsePergunta = executarChat(pergunta).choices[0].message.content

print(responsePergunta)

{
  "1": {
    "tipo": "Gráfico de barras",
    "descrição": "Um gráfico de barras é uma ótima opção para comparar o desempenho de diferentes lojas em vendas de smartphones Apple ao longo de um período de tempo. Este tipo de visualização permite que os usuários vejam facilmente as diferenças nas vendas entre as lojas, e também permite a comparação de vendas ao longo do ano.",
    "variaveis": ["Loja (variável categórica) - para representar as diferentes lojas", "Vendas (variável numérica) - para representar o número de vendas de smartphones Apple", "Mês/Ano (variável temporal) - para representar o período de tempo das vendas"]
  },
  "2": {
    "tipo": "Gráfico de linhas",
    "descrição": "Um gráfico de linhas pode ser útil para visualizar a tendência das vendas de smartphones Apple ao longo do último ano em cada loja. É uma boa opção para mostrar a evolução das vendas em cada loja, identificar picos ou quedas e comparar as performances ao longo do tempo.",
    "variaveis": ["Loja (va

- Guardando as informações da pergunta em um JSON

In [25]:
import json

if responsePergunta is not None:
  jsonConverted = json.loads(responsePergunta)
  with open("responsePergunta.json", "w") as outfile:
    json.dump(jsonConverted, outfile)
  print("Dados salvos em responsePergunta.json")

Dados salvos em responsePergunta.json


- Convertento pergunta para CSV

In [39]:
import csv

with open("responsePergunta.json", "r") as inputfile:
    datas = json.load(inputfile)

# Abrir um arquivo CSV para escrita
with open('responsePergunta.csv', 'w', newline='', encoding='utf-8') as file:
    # Criar um objeto escritor CSV
    csv_writer = csv.writer(file)
    print('Arquivo criado')
    
    # Escrever a linha de cabeçalho
    header = ['Pergunta', 'Tipo', 'Descrição', 'Variveis']
    csv_writer.writerow(header)
    print('Cabeçalho criado')
    
    # Escrever as linhas de dados
    for data in datas:
        row = [promptPergunta, datas[data]['tipo'], datas[data]['descrição'], datas[data]['variaveis']]
        csv_writer.writerow(row)
    print('Informações preenchidas')

Arquivo criado
Cabeçalho criado
Informações preenchidas


- Definindo a função do assistente para avaliar as respostas

In [36]:
resposta = [
    {
        # aqui definimos que a resposta deve vir no formato JSON
        "role": "system",
        "content": "Você como um assistente no processo de design de visualização de dados, deverá avaliar as possibilidades de resposta e responder com uma classificação de qual seria a mais indicada em formato JSON"
    },
    {
        # aqui definimos o padrão de resposta em cada item da lista
        "role": "user",
        "content": "Cada item da lista de sugestões deverá receber uma posição sendo a primeira a melhor e responder no seguinte formato: {/tipo: [tipo da visualuação de dados], nota: [uma nota de 1-7 para a visualização], motivo: [descrever o motivo da nota em relação as demais opções] }/"
    },
    {
        # aqui definimos que a pergunta como a linha escrita no prompt
        "role": "user",
        "content": 'pergunta: ' + promptPergunta + ', avalie as seguintes opções: ' + responsePergunta # type: ignore
     }
]

print(resposta)

[{'role': 'system', 'content': 'Você como um assistente no processo de design de visualização de dados, deverá avaliar as possibilidades de resposta e responder com uma classificação de qual seria a mais indicada em formato JSON'}, {'role': 'user', 'content': 'Cada item da lista de sugestões deverá receber uma posição sendo a primeira a melhor e responder no seguinte formato: {/tipo: [tipo da visualuação de dados], nota: [uma nota de 1-7 para a visualização], motivo: [descrever o motivo da nota em relação as demais opções] }/'}, {'role': 'user', 'content': 'pergunta: Eu, como um gerente de tecnologia, quero mostrar as lojas que mais vendem smartphones apple durante o ultimo ano, quais poderiam ser as melhores visualizações de dados para isso?, avalie as seguintes opções: {\n  "1": {\n    "tipo": "Gráfico de barras",\n    "descrição": "Um gráfico de barras é uma ótima opção para comparar o desempenho de diferentes lojas em vendas de smartphones Apple ao longo de um período de tempo. Est

- Executando o chat com o modelo e o formato para avaliar as respostas

In [37]:
responseResposta = executarChat(resposta).choices[0].message.content

print(responseResposta)

{
  "1": {
    "tipo": "Gráfico de barras",
    "nota": 7,
    "motivo": "O gráfico de barras permite uma comparação clara entre as lojas, evidenciando as diferenças nas vendas de smartphones Apple ao longo do ano. Além disso, a comparação é fácil de ser feita e as tendências podem ser identificadas rapidamente."
  },
  "2": {
    "tipo": "Gráfico de linhas",
    "nota": 5,
    "motivo": "O gráfico de linhas pode ser útil para mostrar a evolução das vendas ao longo do ano, mas pode não ser tão eficiente na comparação entre as lojas, pois a ênfase está na tendência temporal e não na comparação direta entre as lojas."
  },
  "3": {
    "tipo": "Mapa de calor",
    "nota": 4,
    "motivo": "Embora o mapa de calor seja útil para visualizar as vendas por região geográfica, pode não ser a melhor opção para comparar o desempenho das lojas ao longo do ano, pois a ênfase está na localização geográfica em vez das vendas."
  }
}


- Guardando as informações de avaliação de resposta em um JSON

In [38]:
if responseResposta is not None:
  jsonConverted = json.loads(responseResposta)
  with open("responseResposta.json", "w") as outfile:
    json.dump(jsonConverted, outfile)
  print("Dados salvos em responseResposta.json")

Dados salvos em responseResposta.json


- Convertendo respostas para CSV

In [40]:
with open("responseResposta.json", "r") as inputfile:
    datas = json.load(inputfile)

# Abrir um arquivo CSV para escrita
with open('responseResposta.csv', 'w', newline='', encoding='utf-8') as file:
    # Criar um objeto escritor CSV
    csv_writer = csv.writer(file)
    print('Arquivo criado')
    
    # Escrever a linha de cabeçalho
    header = ['Pergunta', 'Tipo', 'Nota', 'Motivo']
    csv_writer.writerow(header)
    print('Cabeçalho criado')
    
    # Escrever as linhas de dados
    for data in datas:
        row = [promptPergunta, datas[data]['tipo'], datas[data]['nota'], datas[data]['motivo']]
        csv_writer.writerow(row)
    print('Informações preenchidas')

Arquivo criado
Cabeçalho criado
Informações preenchidas
